<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">自律型トレーダー</h2>
            <span style="color:#ff7800;">
                MCPサーバーのツールとリソースを活用した自律型エージェントの動作を示す株式取引シミュレーション。
            </span>
        </td>
    </tr>
</table>

### 第6週、4日目へようこそ！

そして今 - キャップストーン・プロジェクトの紹介：

# 自律トレーダー

4人のトレーダーと1人のリサーチャーによる株式取引シミュレーション。

（各トレーダーにリサーチャーがツールとしてアシストに付く感じ）

ツールとリソースを備えた多数のMCPサーバーを活用。

1. accounts.py、accounts_server.py  
自作の account MCPサーバー（エンジニアリング・チームによって書かれています！）

2. Brave Search API  
Brave Software が提供する検索サービス Brave Search を利用するAPI。

3. mcp-server-fetch  
ローカル・ヘッドレス・ブラウザを介してWebページを取得

4. メモリ（sqlite3）  
調査した内容は、sqlite3に永続化して記憶する。

5. market.py、market_server.py  
自作の market MCPサーバー（Polygon.ioで財務データをチェックする）

など、トレーダーのアカウントと、その投資戦略に関する情報を読むためのリソース。

今日のラボの目標は、新しいPythonモジュール `traders.py` を作成することです。

ラボで実験して探索し、準備ができたらPythonモジュールに移行します。

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">もう一度言いますが、</h2>
            <span style="color:#ff7800;">
                実際の取引の判断には使用しないでください。
            </span>
        </td>
    </tr>
</table>

In [1]:
# import

# 基本
import os
from datetime import datetime
from dotenv import load_dotenv

# agents
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio

# 自作
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

from IPython.display import Markdown, display

load_dotenv(override=True)

True

### トレーダーのMCPサーバ
トレーダー・エージェントのためにMCPパラメーションを集めることから始めましょう

In [2]:
# Polygon.io
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
# Polygon.io

if is_paid_polygon or is_realtime_polygon:
    # 有料の場合は公式MCPサーバ
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    # 無料の場合は自作MCPサーバ
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

# アカウント管理、プッシュ通知の自作MCPサーバ
trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### リサーチャーのMCPサーバ
そして、リサーチャー・エージェントのために

In [4]:
# Brave Search API
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

# mcp-server-fetch、mcp/server-brave-search
researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### 次に、それぞれにMcPserverstDioを作成します

In [5]:
# トレーダーのMCPサーバ
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
# リサーチャーのMCPサーバ
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]

# エージェントのMCPサーバ
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### リサーチャー・エージェントを作成し...
- それでは、市場調査を行うためのリサーチャー・エージェントを作ってみましょう。
- そして、それをツールに変えてください - これがOpenai Agents SDKでどのように機能するか、そしてハンドオフとの違いを覚えていますか？

#### リサーチャー・エージェントを作る関数

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    # あなたは金融リサーチャーです。ウェブ上で興味深い金融ニュースを検索し、取引機会を探し、調査に協力することができます。
    # 依頼に基づいて必要な調査を行い、調査結果を報告します。
    # 時間をかけて複数の検索を行い、包括的な概要を把握した上で、調査結果を要約します。
    # 具体的な依頼がない場合は、最新のニュース検索に基づいた投資機会を報告します。
    # 現在の日時は {datetime.now().strftime("%Y-%m-%d %H:%M:%S")} です。
    instructions = f"""
    You are a financial researcher. You are able to search the web for interesting financial news,
    look for possible trading opportunities, and help with research.
    Based on the request, you carry out necessary research and respond with your findings.
    Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
    If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
    The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    """
    
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

#### リサーチャー・エージェントを関数に変換する関数 

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name="Researcher",
        # このツールは、特定の銘柄を調べるという具体的なリクエストに基づいて、
        # または一般的に注目すべき金融ニュースや機会について、
        # オンラインでニュースや機会を調査します。
        # どのような調査を探しているのか説明してください。
        tool_description="This tool researches online for news and opportunities, \
        either based on your specific request to look into a certain stock, \
        or generally for notable financial news and opportunities. \
        Describe what kind of research you're looking for."
    )

#### リサーチャー・エージェントを実行

In [8]:
# Amazonの最新ニュースは何ですか？
research_question = "What's the latest news on Amazon?"

# リサーチャー・エージェントのMCPサーバーを初期化・接続
for server in researcher_mcp_servers:
    await server.connect()

# MCPサーバーを仕込んだリサーチャー・エージェントを取得
researcher = await get_researcher(researcher_mcp_servers)

# リサーチャー・エージェントを単独実行
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)

# 結果の表示
display(Markdown(result.final_output))

Here are some of the latest news highlights about Amazon as of September 2025:

1. Amazon now offers same-day perishable grocery delivery in over 1,000 cities and towns, with plans to double that reach by the end of the year. Same-day delivery is free for orders over $25 for Prime members in most cities.
2. Amazon Prime Day 2025 was the biggest ever, breaking records for sales volume and total items sold during the 4-day event.
3. Amazon's Project Kuiper is progressing with more than 80 satellite missions launched to deploy its initial low Earth orbit satellite internet constellation, currently growing to over 100 satellites.
4. AWS (Amazon Web Services) announced upcoming events including AWS Transform and re:Invent 2025 with innovations and expert-led discussions.
5. Amazon continues to focus on innovation and operational excellence guided by principles such as customer obsession and long-term thinking.

If you want detailed information on any of these topics or the latest stock updates, let me know!

### トレースを見てください

https://platform.openai.com/traces

In [9]:
# あなたはニュースや市場の状況に基づいて積極的に株式を売買するデイトレーダーです。
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."

# 以下、直書きとMCPクライアント混在しとるやん...

# Edの
Account.get("Ed").reset(ed_initial_strategy)
# アカウント情報
display(Markdown(await read_accounts_resource("Ed")))
# 投資戦略の取得
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-15 19:03:17", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### トレーダー・エージェントを作成

In [10]:
# トレーダー・エージェント名は、そのまま取引アカウント名になる。
agent_name = "Ed"

# MCPサーバーを使用してリソースを読み取り
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

# あなたは株式ポートフォリオを管理するトレーダーです。あなたの名前は{agent_name}で、アカウントも{agent_name}です。
# あなたは、インターネットで企業ニュースを検索したり、株価を確認したり、株式を売買したりできるツールにアクセスできます。
# あなたのポートフォリオの投資戦略は： {strategy}
# 現在の保有資産と残高は： {account_details}
# あなたは、関連ニュースや情報をウェブ検索するためのツールを持っています。
# あなたは、株価を確認するためのツールを持っています。
# あなたは、株式を売買するためのツールを持っています。
# あなたは、企業、調査、そしてこれまでの考えを保存するためのツールを持っています。
# これらのツールを活用してポートフォリオを管理してください。
# 取引は、指示を待ったり、確認を求めたりすることなく、ご自身の判断で行ってください。
instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

# ツールを活用してポートフォリオに関する意思決定を行いましょう。
# ニュースや市場を調査し、意思決定を行い、取引を行い、その結果をまとめます。
prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
# instructionsを確認
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-15 19:03:17", 10000.0], ["2025-09-15 19:03:43", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please

### トレーダー・エージェントを実行

In [12]:
# トレーダー・エージェントのMCPサーバーを初期化・接続
for server in mcp_servers:
    await server.connect()

# ツール化したリサーチャー・エージェントを仕込んだトレーダー・エージェントを作成
researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)

# トレーダー・エージェント（サーチャー・エージェント）を実行
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)

# 結果の表示
display(Markdown(result.final_output))

### Summary of Actions Taken:

1. **Market Analysis**:
   - Reviewed the latest stock market conditions, focusing on midcap growth stocks and companies with strong insider ownership.
   - Noted that U.S. markets are at record highs, with strong potential for volatility following economic data releases.

2. **Stock Selection**:
   - Chose to invest in the following midcap growth stocks based on their potential and market conditions:
     - **Monday.com Ltd. (MNDY)**
     - **Wix.com Ltd. (WIX)**
     - **Dolby Laboratories (DLB)**
     - **Krystal Biotech (KRYS)**
     - **Harmony Biosciences Holdings (HRMY)**

3. **Purchases Executed**:
   - Attempted to buy:
     - **50 shares** of **MNDY** at approximately $192.33 per share.
     - **30 shares** of **WIX,** **40 shares** of **DLB,** **25 shares** of **KRYS,** and **20 shares** of **HRMY**.
   - **Result**: 
     - The purchase of **50 shares of MNDY** was successful, reducing the cash balance from $10,000 to approximately $9,980.80.
     - **Subsequent purchases** of WIX, DLB, KRYS, and HRMY failed due to insufficient funds.

### Current Portfolio Status:
- **Holdings**:
  - **MNDY**: 50 shares
- **Cash Balance**: $383.31
- **Portfolio Value**: $9,980.80
- **Total Profit/Loss**: -$19.19

### Next Steps:
- Monitor the performance of MNDY and assess further trading opportunities.
- Consider liquidating positions if there are favorable market conditions.
- Look for new investment opportunities that align with day trading strategy and are within the remaining cash balance. 

If you have specific stocks in mind or would like to analyze other sectors, please let me know!

### その後、トレースを見てください

http://platform.openai.com/traces

In [13]:
# そして、取引の結果を見てみましょう
await read_accounts_resource(agent_name)

'{"name": "ed", "balance": 383.3050000000003, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"MNDY": 50}, "transactions": [{"symbol": "MNDY", "quantity": 50, "price": 192.3339, "timestamp": "2025-09-15 19:05:19", "rationale": "Midcap growth stock with strong potential and recent positive news."}], "portfolio_value_time_series": [["2025-09-15 19:03:17", 10000.0], ["2025-09-15 19:03:43", 10000.0], ["2025-09-15 19:05:20", 9980.805], ["2025-09-15 20:09:03", 9980.805]], "total_portfolio_value": 9980.805, "total_profit_loss": -19.19499999999971}'

### では、これを元に作成したPythonモジュールを確認してみましょう。

- `mcp_params.py` はMCPサーバーを指定する場所です。メモリとプッシュ通知といったおなじみの要素も追加されているのがお分かりいただけるでしょう。

- `templates.py`は、`templates.py` は、指示とメッセージ（システムプロンプトとユーザープロンプト）を設定する場所です。

- `traders.py` はこれらすべてをまとめます。

次のようなコードで少し凝った処理をしていることに気付くでしょう。:

```python
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

これは、コンテキストマネージャーと呼ばれる「with」ステートメントを簡潔に組み合わせた方法であり、以下のような面倒な処理を省くことができます。

```python
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

しかし、それは同等です。

In [14]:
from traders import Trader

trader = Trader("Ed")

await trader.run()

await read_accounts_resource("Ed")

'{"name": "ed", "balance": 39.759280000000274, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"MNDY": 50, "PLTR": 2}, "transactions": [{"symbol": "MNDY", "quantity": 50, "price": 192.3339, "timestamp": "2025-09-15 19:05:19", "rationale": "Midcap growth stock with strong potential and recent positive news."}, {"symbol": "PLTR", "quantity": 2, "price": 171.77286, "timestamp": "2025-09-15 20:10:55", "rationale": "Strong revenue and earnings growth, AI integration."}], "portfolio_value_time_series": [["2025-09-15 19:03:17", 10000.0], ["2025-09-15 19:03:43", 10000.0], ["2025-09-15 19:05:20", 9980.805], ["2025-09-15 20:09:03", 9980.805], ["2025-09-15 20:09:45", 9980.805], ["2025-09-15 20:10:55", 9980.11928], ["2025-09-15 20:11:08", 9980.11928]], "total_portfolio_value": 9980.11928, "total_profit_loss": -19.880719999999258}'

### 今、トレースを見てください

https://platform.openai.com/traces

### 合計でいくつのツールを使用しましたか？

In [15]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools


### 上記で触れらていないファイルの説明

- `push_server.py`はPushoverによるプッシュ通知の自作MCPサーバ
- `templates.py`は、エージェントのシステム・プロンプト定義
- `reset.py`は、（トレーダーの）アカウントの初期化（特に戦略定義）
- `tracers.py`は、トレースとスパンの開始・終了をログに記録

以下は、`5_lab5_ja.ipynb` で触れる内容。

- `util.py`はUI装飾
- `app.py`はエントリポイント